How singular values $\sigma_1,\ \sigma_2$ change with parameters $a,\ b$

In [2]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D

In [10]:
# dissect Jt matrix to rank-1 and rank-1
J,dJ = np.zeros((2,2)),np.zeros((2,2))
JE,JI,a,b=1.6,0.8,0.6,0.150#0.030,0.2#0.01,0.02
# # 1-JE+JI+a+b+2*(a*JI-b*JE)=0
# b = (1-JE+JI+a*(1+2*JI))/(2*JE-1)
# # if aJE+bJI=0
# b=-a*JE/JI
J[:,0],J[:,1]=JE,-JI
dJ[0,0],dJ[1,0]=-a,a
dJ[0,1],dJ[1,1]=b,-b
Jt=J+dJ
print('bJE:',b*JE,'; aJI:',a*JI)
# m and n
m,n=np.array([[1,1],[1,-1]]),np.array([[JE,-a],[-JI,b]])
m1,m2=np.reshape(m[:,0],(2,1)),np.reshape(m[:,1],(2,1))
n1,n2=np.reshape(n[:,0],(2,1)),np.reshape(n[:,1],(2,1))
# standard SVD
lvec,sigv,rvech=la.svd(Jt)
sigv_theo=np.zeros(2,dtype=complex)
# if b*JE>=a*JI:
#     sumsigv=np.sqrt(2*((JE+b)**2+(JI-a)**2))
#     diffsigv=np.sqrt(2*((JE-b)**2+(JI+a)**2))
# elif b*JE<a*JI:
#     sumsigv=np.sqrt(2*((JE-b)**2+(JI+a)**2))
#     diffsigv=np.sqrt(2*((JE+b)**2+(JI-a)**2))

sumsigv=np.sqrt(2*((JE+b)**2+(JI-a)**2))
diffsigv=np.sqrt(2*((JE-b)**2+(JI+a)**2))
sigv_theo[0],sigv_theo[1]=(sumsigv+diffsigv)/2.0,(sumsigv-diffsigv)/2.0

print('num \sigma:',sigv)
print('theo \sigma:',sigv_theo)


bJE: 0.24 ; aJI: 0.48
num \sigma: [2.67071115 0.17972741]
theo \sigma: [ 2.67071115+0.j -0.17972741+0.j]


In [140]:
# dissect Jt matrix to rank-1 and rank-1
J,dJ = np.zeros((2,2)),np.zeros((2,2))
JE,JI,a,b=1.6,0.8,0.2,0.150#0.030,0.2#0.01,0.02
# # 1-JE+JI+a+b+2*(a*JI-b*JE)=0
# b = (1-JE+JI+a*(1+2*JI))/(2*JE-1)
# # if aJE+bJI=0
# b=-a*JE/JI
J[:,0],J[:,1]=JE,-JI
dJ[0,0],dJ[1,0]=-a,a
dJ[0,1],dJ[1,1]=b,-b
Jt=J+dJ
# m and n
m,n=np.array([[1,1],[1,-1]]),np.array([[JE,-a],[-JI,b]])
m1,m2=np.reshape(m[:,0],(2,1)),np.reshape(m[:,1],(2,1))
n1,n2=np.reshape(n[:,0],(2,1)),np.reshape(n[:,1],(2,1))
# standard SVD
lvec,sigv,rvech=la.svd(Jt)
sigmav=np.zeros((2,2),dtype=complex)
sigmavinv= np.zeros((2,2),dtype=complex)
for i in range(2):
    sigmav[i,i]=sigv[i]
    sigmavinv[i,i]=1/sigv[i]
rvec=rvech.T
mlvec=lvec.copy()

# rotating matrix
mlvec[:,0],mlvec[:,1]=mlvec[:,0]*sigv[0],mlvec[:,1]*sigv[1]
rotm=mlvec@np.array([[1/sigv[0],0],[0,1/sigv[1]]])*np.sqrt(2)
invm=la.inv(m)
Mcheck=invm@rotm

# overlap matrix of general-model and svd-model(ortho)
genover,svdover=n.T@m,rvech@mlvec
geneigv,geneigvec=la.eig(genover)
svdeigv,svdeigvec=la.eig(svdover)

# figure out how the dynamics of kappa under different basis vectors
# the derivative of the transfer function equals to the eigenvalue of the overlap matrix
# the identical eigenvalues for overlap matrix nTm and NTM
eigv,eigvec=la.eig(rvech@mlvec)
eigv_,eigvec_=la.eig(n.T@m) 
# the derivative of the transfer function equals to one of the eigenvalues
phid = 1/eigv[0]

# the dynamics of kappa under basis vectors $M$ and $m$
# the initial network activity
x0 = np.reshape(np.array([0.2,-0.05]),(2,1))
inikappa_M = la.inv(m)@x0
# inikappa_M = np.reshape(0.6*geneigvec[:,0],(2,1))
project0=np.reshape(inikappa_M,(2,1))
project0=la.inv(geneigvec)@project0
inikappa_m = la.inv(mlvec)@x0

# main iteration
tf,dt=20,0.001
nt = int(tf/dt)
tt = np.arange(nt)*dt
kappa_M,kappa_M_,kappa_m,kappa_m_=np.zeros((2,nt),dtype=complex),np.zeros((2,nt),dtype=complex),np.zeros((2,nt),dtype=complex),np.zeros((2,nt),dtype=complex)
kappa_M[:,0],kappa_m[:,0]=inikappa_M[:,0],inikappa_m[:,0]
kappa_m_[:,0],kappa_M_[:,0]=sigmavinv@Mcheck.T@kappa_M[:,0],-kappa_M[:,0]
for i in range(1,nt):
    dkappa_M=-kappa_M[:,i-1]+np.squeeze(phid*genover@np.reshape(kappa_M[:,i-1],(2,1)))
    dkappa_M_=-kappa_M_[:,i-1]+np.squeeze(phid*genover@np.reshape(kappa_M_[:,i-1],(2,1)))
    dkappa_m=-kappa_m[:,i-1]+np.squeeze(phid*svdover@np.reshape(kappa_m[:,i-1],(2,1)))

    kappa_M[:,i]=dkappa_M*dt+kappa_M[:,i-1]
    kappa_M_[:,i]=dkappa_M_*dt+kappa_M_[:,i-1]
    kappa_m_[:,i]=sigmavinv@Mcheck.T@kappa_M[:,i]#
    kappa_m[:,i]=dkappa_m*dt+kappa_m[:,i-1]
# line cross the end point of kappa_m represents the slope
slopem=np.zeros((2,2))
slopem[:,0]=[0,0]
slopem[:,1]=kappa_m[:,-1]

# check the distance along the direction of eigenvector
effLambda=np.eye(2)
effLambda[1,1]=0
Eqkappa = (geneigvec@effLambda@la.inv(geneigvec))@np.reshape(inikappa_M,(2,1))
print('Equilibrium:',Eqkappa,' or ',kappa_M[:,-1])
print('l.h.s.:',la.inv(geneigvec)@np.reshape(kappa_M[:,-1],(2,1)))


Equilibrium: [[ 0.4113242 ]
 [-0.01989112]]  or  [ 0.4113242 +0.j -0.01989112+0.j]
l.h.s.: [[4.11804869e-01+0.j]
 [7.93629051e-13+0.j]]


In [141]:
fig = plt.figure()
ax0 = fig.add_subplot(121,projection='3d')
ax0.plot3D(tt,kappa_m[0,:],kappa_m[1,:],'r',label=r'$m$ basis')
ax0.plot3D(tt,kappa_m_[0,:],kappa_m_[1,:],'r--',label=r'transfer from $M$ basis')
ax0.plot3D(np.array([tt[-1],tt[-1]]),slopem[0,:],slopem[1,:],'b*-')
# ax0.ylim([])
ax0.set_xlabel('Time',fontsize=16)
ax0.set_ylabel(r'$\kappa_1$',fontsize=16)
ax0.set_zlabel(r'$\kappa_2$',fontsize=16)
ax0.set_title('dynamics under singular vectors.',fontsize=16)
plt.legend(fontsize=14)
ax1 = fig.add_subplot(122,projection='3d')
ax1.plot3D(tt,kappa_M[0,:],kappa_M[1,:],'r',label=r'$M$ basis')
ax1.set_xlabel('Time',fontsize=16)
ax1.set_ylabel(r'$\kappa_1$',fontsize=16)
ax1.set_zlabel(r'$\kappa_2$',fontsize=16)
ax1.set_title(r'dynamics under $M$ and $N$ vectors ',fontsize=16)
# ax0=ax[0].plot3D(tt,np.real(kappa_m_[0,:]),np.imag(kappa_m_[0,:]),'r--')
# ax0=ax[0].plot3D(tt,np.real(kappa_m_[1,:]),np.imag(kappa_m_[1,:]),'b--')
kappa_Eqm,kappa_EqM=kappa_m[:,-1],kappa_M[:,-1]
plt.legend(fontsize=14)
plt.show()

In [142]:
fig = plt.figure()
ax0 = fig.add_subplot(121)
ax0.plot(kappa_m[0,:],kappa_m[1,:],'r',label=r'$\kappa_1$')
ax0.plot(kappa_m_[0,:],kappa_m_[1,:],'b',label=r'$\kappa_1_$')
ax0.set_xlabel(r'$\kappa_1$')
ax0.set_ylabel(r'$\kappa_2$')
ax0.set_title('dynamics under singular vectors.')
ax1 = fig.add_subplot(122)
ax1.plot(kappa_M[0,:],kappa_M[1,:],'r',label=r'$\kappa_1$')
ax1.set_xlabel(r'$\kappa_1$')
ax1.set_ylabel(r'$\kappa_2$')
ax1.set_title('dynamics under singular vectors.')
plt.legend()
# ax1 = fig.add_subplot(212,projection='3d')
# ax1.plot3D(tt,np.real(kappa_M[0,:]),np.imag(kappa_M[0,:]),'r',label=r'$\kappa_1$')
# ax1.plot3D(tt,np.real(kappa_M[1,:]),np.imag(kappa_M[1,:]),'b',label=r'$\kappa_2$')

In [143]:
fig = plt.figure()
ax0 = fig.add_subplot(121,projection='3d')
ax0.plot3D(tt,np.real(kappa_m[0,:]),np.imag(kappa_m[0,:]),'r',label=r'$\kappa_1$')
ax0.plot3D(tt,np.real(kappa_m[1,:]),np.imag(kappa_m[1,:]),'b',label=r'$\kappa_2$')
ax0.plot3D(tt,np.real(kappa_m_[0,:]),np.imag(kappa_m_[0,:]),'r--',label=r'$\kappa_1$, transfer')
ax0.plot3D(tt,np.real(kappa_m_[1,:]),np.imag(kappa_m_[1,:]),'b--',label=r'$\kappa_2$, transfer')
ax0.set_xlabel('Time',fontsize=16)
ax0.set_ylabel('Real part',fontsize=16)
ax0.set_zlabel('Imag part',fontsize=16)
ax0.set_title('dynamics under singular vectors.',fontsize=16)
plt.legend(fontsize=14)
ax1 = fig.add_subplot(122,projection='3d')
ax1.plot3D(tt,np.real(kappa_M[0,:]),np.imag(kappa_M[0,:]),'r',label=r'$\kappa_1$')
ax1.plot3D(tt,np.real(kappa_M[1,:]),np.imag(kappa_M[1,:]),'b',label=r'$\kappa_2$')
ax1.set_xlabel('Time',fontsize=16)
ax1.set_ylabel('Real part',fontsize=16)
ax1.set_zlabel('Imag part',fontsize=16)
ax1.set_title(r'dynamics under $M$ and $N$ vectors ',fontsize=16)
# ax0=ax[0].plot3D(tt,np.real(kappa_m_[0,:]),np.imag(kappa_m_[0,:]),'r--')
# ax0=ax[0].plot3D(tt,np.real(kappa_m_[1,:]),np.imag(kappa_m_[1,:]),'b--')
kappa_Eqm,kappa_EqM=kappa_m[:,-1],kappa_M[:,-1]
plt.legend(fontsize=14)
plt.show()

In [144]:
# We give different initial kappa0 and find the fixed point(line) under same condition with identical eigenvalues and overlap matrix
tf,dt=5,0.01
nt = int(tf/dt)
tt = np.arange(nt)*dt
kmax,dk=0.5,0.1
nstep=int(2*kmax/dk)
genv,genvec=la.eig(genover)
Vinv = la.inv(genvec)
firstrowinv = np.reshape(Vinv[0,:],(1,2))
distance_phase=np.zeros((nstep,nstep))
# generate phase plane 
kappaM1,kappaM2=np.linspace(-kmax,kmax,nstep),np.linspace(-kmax,kmax,nstep)
kappaM_phase=np.zeros((nstep,nstep,2,nt),dtype=complex)
rho_phase=np.zeros((nstep,nstep,2),dtype=complex)
for idxk1, k1 in enumerate(kappaM1):
    for idxk2, k2 in enumerate(kappaM2):
        inikappa_M = np.array([k1,k2])
        kappaM_phase[idxk1,idxk2,:,0]=inikappa_M[:]
        for i in range(1,nt):
            dkappa_M=-kappaM_phase[idxk1,idxk2,:,i-1]+np.squeeze(phid*genover@np.reshape(kappaM_phase[idxk1,idxk2,:,i-1],(2,1)))
            kappaM_phase[idxk1,idxk2,:,i]=dkappa_M*dt+kappaM_phase[idxk1,idxk2,:,i-1]
        rho_phase[idxk1,idxk2,:]=kappaM_phase[idxk1,idxk2,:,-1]/genvec[:,0]
        print(firstrowinv@(np.reshape(inikappa_M,(2,1))))
        distance_phase[idxk1,idxk2]=np.squeeze(firstrowinv@(np.reshape(inikappa_M,(2,1))))



[[-1.87277173]]
[[-1.58190705]]
[[-1.29104236]]
[[-1.00017767]]
[[-0.70931298]]
[[-0.4184483]]
[[-0.12758361]]
[[0.16328108]]
[[0.45414576]]
[[0.74501045]]
[[-1.74746492]]
[[-1.45660024]]
[[-1.16573555]]
[[-0.87487086]]
[[-0.58400618]]
[[-0.29314149]]
[[-0.0022768]]
[[0.28858789]]
[[0.57945257]]
[[0.87031726]]
[[-1.62215812]]
[[-1.33129343]]
[[-1.04042874]]
[[-0.74956405]]
[[-0.45869937]]
[[-0.16783468]]
[[0.12303001]]
[[0.4138947]]
[[0.70475938]]
[[0.99562407]]
[[-1.49685131]]
[[-1.20598662]]
[[-0.91512193]]
[[-0.62425724]]
[[-0.33339256]]
[[-0.04252787]]
[[0.24833682]]
[[0.5392015]]
[[0.83006619]]
[[1.12093088]]
[[-1.3715445]]
[[-1.08067981]]
[[-0.78981512]]
[[-0.49895044]]
[[-0.20808575]]
[[0.08277894]]
[[0.37364363]]
[[0.66450831]]
[[0.955373]]
[[1.24623769]]
[[-1.24623769]]
[[-0.955373]]
[[-0.66450831]]
[[-0.37364363]]
[[-0.08277894]]
[[0.20808575]]
[[0.49895044]]
[[0.78981512]]
[[1.08067981]]
[[1.3715445]]
[[-1.12093088]]
[[-0.83006619]]
[[-0.5392015]]
[[-0.24833682]]
[[0.0425278

In [145]:
fig = plt.figure()
ax0 = fig.add_subplot(121,projection='3d')
for i in range(nstep):
  for j in range(nstep):
    ax0.plot3D(tt,kappaM_phase[i,j,0,:],kappaM_phase[i,j,1,:])
    # ax0.plot3D(tt,kappam_phase[i,j,0,:],kappam_phase[i,j,1,:])
    # ax0.plot3D(tt,np.real(kappaM_phase[i,j,1,:]),np.imag(kappaM_phase[i,j,1,:]),'b')
    # ax1=ax[0,1].plot(np.imag(kappaM_phase[i,j,0,:]),np.imag(kappaM_phase[i,j,1,:]))
    # ax2=ax[1,0].plot(np.real(rho_phase[i,j,0]),np.imag(rho_phase[i,j,0]),'k.')
    # ax3=ax[1,1].plot(np.real(rho_phase[i,j,1]),np.imag(rho_phase[i,j,1]),'k.')
ax0.plot3D(np.array([tt[-1],tt[-1]]),[0,geneigvec[0,0]],[0,geneigvec[1,0]],'b*-')
ax0.set_xlabel('Time')
ax0.set_ylabel(r'$\kappa_1$')
ax0.set_zlabel(r'$\kappa_2$')
ax0.set_title('dynamics after different initiations')

ax1 = fig.add_subplot(122)
ax1.set_xticklabels([-0.6,-0.4,-0.2,0.0,0.2,0.4,0.6])
ax1.set_yticklabels([-0.6,-0.4,-0.2,0.0,0.2,0.4,0.6])
ax1.set_xlabel(r'$\kappa_1$')
ax1.set_ylabel(r'$\kappa_2$')
h1=ax1.imshow(distance_phase,cmap='jet', interpolation='spline16',alpha=0.85)
ax1.set_title(r'distance along $u_1$')

divider = make_axes_locatable(ax1)
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(h1, cax=cax, orientation='vertical');

In [146]:
fig = plt.figure()
ax0 = fig.add_subplot(211,projection='3d')
for i in range(nstep):
  for j in range(nstep):
    ax0.plot3D(tt,np.real(kappaM_phase[i,j,0,:]),np.imag(kappaM_phase[i,j,0,:]),'r')
    # ax0.plot3D(tt,np.real(kappaM_phase[i,j,1,:]),np.imag(kappaM_phase[i,j,1,:]),'b')
    # ax1=ax[0,1].plot(np.imag(kappaM_phase[i,j,0,:]),np.imag(kappaM_phase[i,j,1,:]))
    # ax2=ax[1,0].plot(np.real(rho_phase[i,j,0]),np.imag(rho_phase[i,j,0]),'k.')
    # ax3=ax[1,1].plot(np.real(rho_phase[i,j,1]),np.imag(rho_phase[i,j,1]),'k.')
# print(rho_phase[:,:,0])

In [147]:
# quiver phase plane
kappaM1,kappaM2=np.linspace(-kmax,kmax,nstep),np.linspace(-kmax,kmax,nstep)
[KAPPAM1,KAPPAM2]=np.meshgrid(kappaM1,kappaM2)

dKAPPAM1 = (-KAPPAM1+phid*(genover[0,0]*KAPPAM1+genover[0,1]*KAPPAM2))
dKAPPAM2 = (-KAPPAM2+phid*(genover[1,0]*KAPPAM1+genover[1,1]*KAPPAM2))

fig, ax = plt.subplots()
ax.quiver(KAPPAM2, KAPPAM1, dKAPPAM2, dKAPPAM1)
ax.set(aspect=1, title=r'diagram of $\kappa_1$ v.s. $\kappa_2$ ', xlabel=r'$\kappa_1$', ylabel=r'$\kappa_2$')

plt.show()

In [148]:
# go back to our two-by-two case
# the two-by-two case
J=np.zeros((2,2))
JE,JI=0.6,1.0
J[:,0],J[:,1]=JE,-JI
a,b=0.0,0.0#0.20,0.18
J[0,0],J[0,1],J[1,0],J[1,1]=J[0,0]-a,J[0,1]+b,J[1,0]+a,J[1,1]-b

# theo solutions
s,p,q=JE**2+JI**2,a**2+b**2,JE*a+JI*b
l=np.sqrt(((JE+b)**2+(JI-a)**2)*((JE-b)**2+(JI+a)**2))
x1,y1=2*q/(s-p+l),(2*(JE*JI+a*b)-l)/(JE**2+a**2-JI**2-b**2)
x2,y2=2*q/(s-p-l),(2*(JE*JI+a*b)+l)/(JE**2+a**2-JI**2-b**2)
lu,sigv,rvh=la.svd(J)
rv=rvh.T
eigv,eigvec=la.eig(J)
print('largest eigenvalue:',eigv[0])

largest eigenvalue: 4.440892098500626e-16


In [149]:
# calculate the theo results
k,alpha,beta=JI/JE,a/JE,b/JE
theo_sig=np.zeros(2,dtype=complex)
theo_sig[0]=np.sqrt(2)*JE*(np.sqrt(1+k**2)+(alpha**2+beta**2-(k*alpha-beta)**2/(k**2+1))/2/np.sqrt(1+k**2))
print('theo sig0:',theo_sig[0],'; num:',sigv[0])
theo_sig[1]=np.sqrt(2)*JE*(beta-alpha*k)/np.sqrt(k**2+1)
print('theo sig1:',theo_sig[1],'; num:',sigv[1])

theo sig0: (1.6492422502470643+0j) ; num: 1.649242250247064
theo sig1: 0j ; num: 4.454148674044418e-17


In [150]:
def eigval_set(JE=1.4,JI=1.2,maxv=0.1,nstep=100):
    aset,bset=np.linspace(-maxv,maxv,2*nstep),np.linspace(-maxv,maxv,2*nstep)
    theo_sigv=np.zeros((2,len(aset),len(bset)))
    nume_sigv=np.zeros_like(theo_sigv)
    J=np.zeros((2,2))
    k=JI/JE
    J[:,0],J[:,1]=JE,-JI
    Jt=np.zeros_like(J)
    for ia, av in enumerate(aset):
        for ib,bv in enumerate(bset):
            alpha,beta=av/JE,bv/JE
            if bv*JE>=av*JI:
                theo_sigv[0,ia,ib]=np.sqrt(2)*JE*(np.sqrt(1+k**2)+(alpha**2+beta**2-(k*alpha-beta)**2/(k**2+1))/2/np.sqrt(1+k**2))
                theo_sigv[1,ia,ib]=np.sqrt(2)*JE*(beta-alpha*k)/np.sqrt(k**2+1)
            else:
                theo_sigv[0,ia,ib]=np.sqrt(2)*JE*(np.sqrt(1+k**2)+(alpha**2+beta**2-(k*alpha-beta)**2/(k**2+1))/2/np.sqrt(1+k**2))
                theo_sigv[1,ia,ib]=np.sqrt(2)*JE*(-beta+alpha*k)/np.sqrt(k**2+1)
                Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=J[0,0]-av,J[0,1]+bv,J[1,0]+av,J[1,1]-bv
                _,nume_sigv[:,ia,ib],_=la.svd(Jt)
    [xs,ys] = np.meshgrid(aset,bset)
    fig,ax = plt.subplots(1,2,figsize=(8,6))
    # ax=ax.flatten()
    ax0=ax[0].scatter(xs,ys,c=theo_sigv[0,:,:],cmap='jet')
    ax1=ax[1].scatter(xs,ys,c=theo_sigv[1,:,:],cmap='jet')
    # ax0=ax[0].scatter(xs,ys,c=theo_sigv[0,:,:],cmap='Blues',vmin=1.1,vmax=1.2)
    # ax1=ax[1].scatter(xs,ys,c=theo_sigv[1,:,:],cmap='Blues',vmin=0.05,vmax=0.15)
    divider0 = make_axes_locatable(ax[0])
    cax0 = divider0.append_axes("right", size="5%", pad=0.05)
    divider1 = make_axes_locatable(ax[1])
    cax1 = divider1.append_axes("right", size="5%", pad=0.05)
    fig.colorbar(ax0,ax=ax[0],cax=cax0)
    fig.colorbar(ax1,ax=ax[1],cax=cax1)
    ax[0].set_title(r'$\sigma_1$')
    ax[1].set_title(r'$\sigma_2$')
    ax[0].axis('square')
    ax[1].axis('square')
    ax[1].set_yticks([])


    return JI/JE
_ = widgets.interact(eigval_set,JE=(0.0,2.0,.1),
                     JI=(0.0,2.0,.1),maxv=[0.1,0.15,0.2],nstep=100)


# hm=plt.imshow(theo_sigv[0,:,:], cmap='Blues',interpolation="nearest")
# plt.xticks(np.arange(0, 2*nstep,100), np.arange(-maxv, maxv, 0.1))
# plt.yticks(np.arange(0, 2*nstep,100), np.arange(-maxv, maxv, 0.1))
# plt.title(r'$\sigma_1$')
# plt.subplot(122)
# hm=plt.imshow(theo_sigv[1,:,:], cmap='Blues',interpolation="nearest")
# plt.xticks(np.arange(0, 2*nstep,100), np.arange(-maxv, maxv, 0.1))
# plt.yticks(np.arange(0, 2*nstep,100), np.arange(-maxv, maxv, 0.1))
# plt.title(r'$\sigma_2$')
# ax1 = fig.add_subplot(121, projection='3d')
# ax1.scatter(xs, ys, (np.squeeze(theo_sigv[0,:,:])), c='r', marker='o')
# ax1.scatter(xs, ys, (np.squeeze(theo_sigv[1,:,:])), c='b', marker='o')
# ax2 = fig.add_subplot(122, projection='3d')
# ax2.scatter(xs, ys, (np.squeeze(nume_sigv[0,:,:])), c='r', marker='o')
# ax2.scatter(xs, ys, (np.squeeze(nume_sigv[1,:,:])), c='b', marker='o')

interactive(children=(FloatSlider(value=1.4, description='JE', max=2.0), FloatSlider(value=1.2, description='J…

Ploting two vectors, showing how the rotating angel changes with parameters a and b

In [151]:
def vectors_rotate(JE=0.8,JI=0.6,a=0.0,b=0.0):
  J=np.zeros((2,2))
  J[:,0],J[:,1]=JE,-JI
  J[0,0],J[0,1],J[1,0],J[1,1]=J[0,0]-a,J[0,1]+b,J[1,0]+a,J[1,1]-b

  # calculate the theo results
  k,alpha,beta=JI/JE,a/JE,b/JE
  lu,sigv,rvh=la.svd(J)
  rv=rvh.T
  tgu_theta=(JE**2+JI**2-a**2-b**2)/((JE+a)**2+(JI+b)**2-2*(k*a-b)**2/(k**2+1))
  tgv_gamma=(JE*JI+a*b)/((k*a-b)**2/(k**2+1)-(JI**2+b**2))
  tan_delta=(-tgu_theta+tgv_gamma)/(1+tgu_theta*tgv_gamma)
  theta_u=np.arctan(tgu_theta)
  gamma_v=np.arctan(tgv_gamma)+np.pi
  right_xend,right_yend=np.cos(gamma_v),np.sin(gamma_v)
  left_xend,left_yend=np.cos(theta_u),np.sin(theta_u)
  theta_u_=np.arctan2(-lu[0,0],-lu[1,0])
  gamma_v_=np.arctan2(-rv[0,0],-rv[1,0])
  


  right_xend_,right_yend_=np.cos(gamma_v_),np.sin(gamma_v_)
  left_xend_,left_yend_=np.cos(theta_u_),np.sin(theta_u_)

  delta_theta= gamma_v-theta_u
  delta_left_xend,delta_left_yend=np.cos(0),np.sin(0)
  delta_right_xend,delta_right_yend=np.cos(delta_theta),np.sin(delta_theta)

  overlap_=lu[:,0].T@rv[:,0]
  overlap=np.cos(gamma_v-theta_u)

  # both left and right vectors are originated from the zero point (0,0)
  left_start=[0,0]
  left_end=[left_xend,left_yend]
  left_end_=[left_xend_,left_yend_]
  delta_left_end=[delta_left_xend,delta_left_yend]
  right_start=[0,0]
  right_end=[right_xend,right_yend]
  right_end_=[right_xend_,right_yend_]
  delta_right_end=[delta_right_xend,delta_right_yend]
  X=[left_start[0],right_start[0]]
  Y=[left_start[1],right_start[1]]

  U=[left_end[0],right_end[0]]
  V=[left_end[1],right_end[1]]
  Ud=[delta_left_end[0],delta_right_end[0]]
  Vd=[delta_left_end[1],delta_right_end[1]]

  U_=[left_end_[0],right_end_[0]]
  V_=[left_end_[1],right_end_[1]]

  fig,ax=plt.subplots(1,2)
  # ax=fig.add_subplot(211)
  ax[0].quiver(X[0],Y[0],U[0],V[0],angles='xy', color = 'r', scale_units='xy', scale=1,label=r'$u^{(1)}$')
  ax[0].quiver(X[1],Y[1],U[1],V[1],angles='xy', color = 'B', scale_units='xy', scale=1,label=r'$v^{(1)}$')
  ax[0].axis('square')
  ax[0].set_xlim([-1.5, 1.5])
  ax[0].set_ylim([-1.5, 1.5])
  ax[0].legend()
  
  ax[1].quiver(X,Y,Ud,Vd,angles='xy', scale_units='xy', scale=1)
  ax[1].axis('square')
  ax[1].set_xlim([-1.5, 1.5])
  ax[1].set_ylim([-1.5, 1.5])
  # ax[1].quiver(X,Y,U_,V_,angles='xy', scale_units='xy', scale=1)
  # ax[1].axis('square')
  # ax[1].set_xlim([-1.5, 1.5])
  # ax[1].set_ylim([-1.5, 1.5])
  plt.show()
  return (overlap,overlap_)
_ = widgets.interact(vectors_rotate,JE=(0.0,2.0,.1),
                     JI=(0.0,2.0,.1),a=(-0.1,0.1,0.02),b=(-0.1,0.1,0.02))

interactive(children=(FloatSlider(value=0.8, description='JE', max=2.0), FloatSlider(value=0.6, description='J…

In [152]:

# go back to our two-by-two case
# the two-by-two case
J=np.zeros((2,2))
maxv,nstep=0.1,100
aset,bset=np.linspace(-maxv,maxv,nstep),np.linspace(-maxv,maxv,nstep)
theo_theta,nume_theta=np.zeros((2,len(aset),len(bset))),np.zeros((2,len(aset),len(bset)))
theo_gamma,nume_gamma=np.zeros((2,len(aset),len(bset))),np.zeros((2,len(aset),len(bset)))
theo_dtheta,nume_dtheta=np.zeros((2,len(aset),len(bset))),np.zeros((2,len(aset),len(bset)))
theo_overlap,nume_overlap=np.zeros((2,len(aset),len(bset))),np.zeros((2,len(aset),len(bset)))

# when JE>JI
JE,JI=1.1,1.0
J[:,0],J[:,1]=JE,-JI
k=JI/JE
Jt=np.zeros_like(J)
for ia, av in enumerate(aset):
  for ib,bv in enumerate(bset):
    # theoretical results
    tgu_theta=(JE**2+JI**2-av**2-bv**2)/((JE+av)**2+(JI+bv)**2-2*(k*av-bv)**2/(k**2+1))
    tgv_gamma=(JE*JI+av*bv)/((k*av-bv)**2/(k**2+1)-(JI**2+bv**2))
    theta_u=np.arctan(tgu_theta)
    gamma_v=np.arctan(tgv_gamma)+np.pi
    delta_theta=(-theta_u+gamma_v)
    theo_theta[0,ia,ib],theo_gamma[0,ia,ib],theo_dtheta[0,ia,ib]=theta_u,gamma_v,delta_theta
    theo_overlap[0,ia,ib] = np.cos(delta_theta)

    # numerical results
    Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=J[0,0]-av,J[0,1]+bv,J[1,0]+av,J[1,1]-bv
    lu,sigv,rvh=la.svd(Jt)
    rv=rvh.T
    theta_u_=np.arctan2(lu[0,0],lu[1,0])
    gamma_v_=np.arctan2(rv[0,0],rv[1,0])
    delta_theta_=(-theta_u_+gamma_v_)
    nume_theta[0,ia,ib],nume_gamma[0,ia,ib],nume_dtheta[0,ia,ib]=theta_u_,gamma_v_,delta_theta_
    nume_overlap[0,ia,ib] = lu[:,0].T@rv[:,0]

# when JE<JI
JE,JI=.9,1.0
J[:,0],J[:,1]=JE,-JI
k=JI/JE
Jt=np.zeros_like(J)
for ia, av in enumerate(aset):
  for ib,bv in enumerate(bset):
    # theoretical results
    tgu_theta=(JE**2+JI**2-av**2-bv**2)/((JE+av)**2+(JI+bv)**2-2*(k*av-bv)**2/(k**2+1))
    tgv_gamma=(JE*JI+av*bv)/((k*av-bv)**2/(k**2+1)-(JI**2+bv**2))
    theta_u=np.arctan(tgu_theta)
    gamma_v=np.arctan(tgv_gamma)+np.pi
    delta_theta=(-theta_u+gamma_v)
    theo_theta[1,ia,ib],theo_gamma[1,ia,ib],theo_dtheta[1,ia,ib]=theta_u,gamma_v,delta_theta
    theo_overlap[1,ia,ib] = np.cos(delta_theta)

    # numerical results
    Jt[0,0],Jt[0,1],Jt[1,0],Jt[1,1]=J[0,0]-av,J[0,1]+bv,J[1,0]+av,J[1,1]-bv
    lu,sigv,rvh=la.svd(Jt)
    rv=rvh.T
    theta_u_=np.arctan2(lu[0,0],lu[1,0])
    gamma_v_=np.arctan2(rv[0,0],rv[1,0])
    delta_theta_=(-theta_u_+gamma_v_)
    nume_theta[1,ia,ib],nume_gamma[1,ia,ib],nume_dtheta[1,ia,ib]=theta_u_,gamma_v_,delta_theta_
    nume_overlap[1,ia,ib] = lu[:,0].T@rv[:,0]
[xs,ys] = np.meshgrid(aset,bset)
fig,ax = plt.subplots(1,2,figsize =(8,6))
ax0=ax[0].scatter(xs,ys,c=theo_overlap[0,:,:],cmap='jet',label=r'$u^{(1)\intercal}v^{(1)}$')
ax[0].axis('square')
ax[0].set_xlabel('a for E')
ax[0].set_ylabel('b for I')
ax[0].set_title(r'$J_E=1.1,\ J_I=1.0$')
ax[0].legend()
ax1=ax[1].scatter(xs,ys,c=theo_overlap[1,:,:],cmap='jet',label=r'$u^{(1)\intercal}v^{(1)}$')
ax[1].axis('square')
ax[1].set_xlabel('a for E')
ax[1].set_ylabel('b for I')
ax[1].set_title(r'$J_E=0.9,\ J_I=1.0$')
ax[1].legend()
divider0 = make_axes_locatable(ax[0])
cax0 = divider0.append_axes("right", size="5%", pad=0.05)
divider1 = make_axes_locatable(ax[1])
cax1 = divider1.append_axes("right", size="5%", pad=0.05)
fig.colorbar(ax0,ax=ax[0],cax=cax0)
fig.colorbar(ax1,ax=ax[1],cax=cax1)


In [153]:
JI,JE=2.76,2.8
k=JI/JE
alpha=np.sqrt((k**2+1)/2.0)-1
a = JE*alpha
b = a*k
J=np.zeros((2,2))
J[:,0],J[:,1]=JE,-JI
J[0,0],J[0,1],J[1,0],J[1,1]=J[0,0]-a,J[0,1]+b,J[1,0]+a,J[1,1]-b
print(1-(JE**2+JI**2)/((JE+a)**2+(JI+b)**2))
print((k**2-1)/(1+k**2),'ab:',a,b)
# calculate the theo results
k,alpha,beta=JI/JE,a/JE,b/JE
lu,sigv,rvh=la.svd(J)
rv=rvh.T
tgu_theta=(JE**2+JI**2-a**2-b**2)/((JE+a)**2+(JI+b)**2-2*(k*a-b)**2/(k**2+1))
tgv_gamma=(JE*JI+a*b)/((k*a-b)**2/(k**2+1)-(JI**2+b**2))
tan_delta=(-tgu_theta+tgv_gamma)/(1+tgu_theta*tgv_gamma)
theta_u=np.arctan(tgu_theta)
gamma_v=np.arctan(tgv_gamma)+np.pi
right_xend,right_yend=np.cos(gamma_v),np.sin(gamma_v)
left_xend,left_yend=np.cos(theta_u),np.sin(theta_u)
theta_u_=np.arctan2(-lu[0,0],-lu[1,0])
gamma_v_=np.arctan2(-rv[0,0],-rv[1,0])



right_xend_,right_yend_=np.cos(gamma_v_),np.sin(gamma_v_)
left_xend_,left_yend_=np.cos(theta_u_),np.sin(theta_u_)

delta_theta= gamma_v-theta_u
delta_left_xend,delta_left_yend=np.cos(0),np.sin(0)
delta_right_xend,delta_right_yend=np.cos(delta_theta),np.sin(delta_theta)

overlap_=lu[:,0].T@rv[:,0]
overlap=np.cos(gamma_v-theta_u)
print('overlap',overlap)

# both left and right vectors are originated from the zero point (0,0)
left_start=[0,0]
left_end=[left_xend,left_yend]
left_end_=[left_xend_,left_yend_]
delta_left_end=[delta_left_xend,delta_left_yend]
right_start=[0,0]
right_end=[right_xend,right_yend]
right_end_=[right_xend_,right_yend_]
delta_right_end=[delta_right_xend,delta_right_yend]
X=[left_start[0],right_start[0]]
Y=[left_start[1],right_start[1]]

U=[left_end[0],right_end[0]]
V=[left_end[1],right_end[1]]
Ud=[delta_left_end[0],delta_right_end[0]]
Vd=[delta_left_end[1],delta_right_end[1]]

U_=[left_end_[0],right_end_[0]]
V_=[left_end_[1],right_end_[1]]

fig,ax=plt.subplots(1,2)
# ax=fig.add_subplot(211)
ax[0].quiver(X[0],Y[0],U[0],V[0],angles='xy', color = 'r', scale_units='xy', scale=1,label=r'$u^{(1)}$')
ax[0].quiver(X[1],Y[1],U[1],V[1],angles='xy', color = 'B', scale_units='xy', scale=1,label=r'$v^{(1)}$')
ax[0].axis('square')
ax[0].set_xlim([-1.5, 1.5])
ax[0].set_ylim([-1.5, 1.5])
ax[0].legend()

ax[1].quiver(X,Y,Ud,Vd,angles='xy', scale_units='xy', scale=1)
ax[1].axis('square')
ax[1].set_xlim([-1.5, 1.5])
ax[1].set_ylim([-1.5, 1.5])


-0.01438774453990277
-0.014387744539902762 ab: -0.019928058484816977 -0.019643371935033876
overlap 0.014310675562345328


(-1.5, 1.5)